In [3]:
import cv2
import dlib
import os
import time
import numpy as np

def resize_image(image: np.ndarray, max_size: int = 800) -> np.ndarray:
    """
    Resize the image while maintaining aspect ratio, so that the maximum dimension is not greater than max_size.
    
    Args:
        - image (numpy.ndarray): Input image.
        - max_size (int): Maximum size for the larger dimension after resizing. Default is 800.

    Returns:
        numpy.ndarray: Resized image.
    """
    if image is None:
        return None
    
    height, width = image.shape[:2]
    if max(height, width) > max_size:
        if height > width:
            new_height = max_size
            new_width = int(width * (max_size / height))
        else:
            new_width = max_size
            new_height = int(height * (max_size / width))
        image = cv2.resize(image, (new_width, new_height))
    return image

def dlib_cnn_detect_faces(source_folder: str, destination_folder: str, model_path: str) -> None:
    """
    Detect faces using DLIB's CNN face detector and save bounding boxes to text files.
    
    Args:
        - source_folder (str): Path to the source directory containing image files.
        - destination_folder (str): Path to the destination directory where processed images and labels will be saved.
        - model_path (str): Path to the pre-trained model for face detection.
    """
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
    
    # Ensure the destination folders exist
    images_folder = os.path.join(destination_folder, "images")
    labels_folder = os.path.join(destination_folder, "labels")
    if not os.path.exists(images_folder):
        os.makedirs(images_folder)
    if not os.path.exists(labels_folder):
        os.makedirs(labels_folder)
    
    # Load the CNN face detector
    cnn_detector = dlib.cnn_face_detection_model_v1(model_path)
    
    # Iterate through all files in the source directory
    for filename in os.listdir(source_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')) and not filename.startswith('.'):
            file_path = os.path.join(source_folder, filename)
            # Load the image
            # Skip files that are not regular image files
            try:
                image = cv2.imread(file_path)
            except:
                print(f"Skipping file: {file_path}")
                continue
            
            # Resize the image while maintaining aspect ratio
            image = resize_image(image)
            
            # Record start time
            start = time.time()
            
            # Perform face detection
            detections = cnn_detector(image, 2)
            
            for i, face in enumerate(detections):
                l, t, r, b = face.rect.left(), face.rect.top(), face.rect.right(), face.rect.bottom()
                
                # Ensure x1, y1, x2, y2 format
                x1, y1, x2, y2 = min(l, r), min(t, b), max(l, r), max(t, b)
                
                # Draw bounding boxes for each detected face
                cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)
                
                # Save bounding box coordinates to a text file
                label_file_path = os.path.join(labels_folder, f"{os.path.splitext(filename)[0]}.txt")
                with open(label_file_path, 'w') as label_file:
                    label_file.write(f"{x1} {y1} {x2} {y2}\n")
            
            # Record end time
            end = time.time()
            
            print(f"Processed {filename} in {end - start:.2f} seconds.")
            
            # Save the processed image to the images folder
            output_image_path = os.path.join(images_folder, filename)
            cv2.imwrite(output_image_path, image)
    print("Processing complete.")


In [1]:
def dlib_cnn_detect_faces_score(source_folder: str, destination_folder: str, model_path: str) -> None:
    """
    Detect faces using DLIB's CNN face detector and save bounding boxes to text files with confidence scores.
    
    Args:
        - source_folder (str): Path to the source directory containing image files.
        - destination_folder (str): Path to the destination directory where processed images and labels will be saved.
        - model_path (str): Path to the pre-trained model for face detection.
    """
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
    
    images_folder = os.path.join(destination_folder, "images")
    labels_folder = os.path.join(destination_folder, "labels")
    os.makedirs(images_folder, exist_ok=True)
    os.makedirs(labels_folder, exist_ok=True)
    
    cnn_detector = dlib.cnn_face_detection_model_v1(model_path)
    
    for filename in os.listdir(source_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')) and not filename.startswith('.'):
            file_path = os.path.join(source_folder, filename)
            try:
                image = cv2.imread(file_path)
            except:
                print(f"Skipping file: {file_path}")
                continue

            image = resize_image(image)
            start = time.time()
            detections = cnn_detector(image, 2)

            label_file_path = os.path.join(labels_folder, f"{os.path.splitext(filename)[0]}.txt")
            with open(label_file_path, 'a') as label_file:  # Use 'a' to append if processing multiple faces
                for i, face in enumerate(detections):
                    l, t, r, b = face.rect.left(), face.rect.top(), face.rect.right(), face.rect.bottom()
                    confidence = face.confidence
                    
                    x1, y1, x2, y2 = min(l, r), min(t, b), max(l, r), max(t, b)
                    cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)
                    
                    # Include confidence value with bounding box coordinates
                    label_file.write(f"{confidence} {x1} {y1} {x2} {y2}\n")

            end = time.time()
            print(f"Processed {filename} in {end - start:.2f} seconds.")
            
            output_image_path = os.path.join(images_folder, filename)
            cv2.imwrite(output_image_path, image)
    print("Processing complete.")

In [12]:
source_folder = "/Users/jocareher/Downloads/test/images"
destination_folder = "/Users/jocareher/Downloads/dlib_predictions"
model_path = "/Users/jocareher/Library/CloudStorage/OneDrive-Personal/Educación/eLearning/Face_Detection/Weights/mmod_human_face_detector.dat"
dlib_cnn_detect_faces(source_folder=source_folder,
                      destination_folder=destination_folder,
                      model_path=model_path)

/Users/jocareher/Downloads/test/images/face_img_481.jpg
Processed face_img_481.jpg in 3.16 seconds.
/Users/jocareher/Downloads/test/images/flip_face_img_2180.jpg
Processed flip_face_img_2180.jpg in 2.86 seconds.
/Users/jocareher/Downloads/test/images/face_img_1214.jpg
Processed face_img_1214.jpg in 2.87 seconds.
/Users/jocareher/Downloads/test/images/face_img_3371.jpg
Processed face_img_3371.jpg in 2.86 seconds.
/Users/jocareher/Downloads/test/images/face_img_7117.jpg
Processed face_img_7117.jpg in 2.85 seconds.
/Users/jocareher/Downloads/test/images/face_img_6584.jpg
Processed face_img_6584.jpg in 2.83 seconds.
/Users/jocareher/Downloads/test/images/face_img_2090.jpg
Processed face_img_2090.jpg in 2.93 seconds.
/Users/jocareher/Downloads/test/images/flip_face_img_2631.jpg
Processed flip_face_img_2631.jpg in 3.02 seconds.
/Users/jocareher/Downloads/test/images/flip_face_bcn_15.jpg
Processed flip_face_bcn_15.jpg in 6.49 seconds.
/Users/jocareher/Downloads/test/images/face_img_697.jpg
Pr

In [5]:
source_folder ="/Users/jocareher/Downloads/benchmark_test_split_rotated/0/images"
destination_folder = "/Users/jocareher/Downloads/dlib_predictions_score/0"
model_path = "/Users/jocareher/Library/CloudStorage/OneDrive-Personal/Educación/eLearning/Face_Detection/Weights/mmod_human_face_detector.dat"
dlib_cnn_detect_faces_score(source_folder=source_folder,
                      destination_folder=destination_folder,
                      model_path=model_path)

Processed flip_face_img_2180.jpg in 3.63 seconds.
Processed face_img_1214.jpg in 4.70 seconds.
Processed face_img_7117.jpg in 3.58 seconds.
Processed face_img_2090.jpg in 2.87 seconds.
Processed flip_face_img_65.jpg in 3.26 seconds.
Processed face_img_1002.jpg in 4.24 seconds.
Processed flip_face_img_548.jpg in 3.65 seconds.
Processed flip_face_bcn_368.jpg in 5.12 seconds.
Processed flip_face_img_1265.jpg in 3.63 seconds.
Processed flip_face_img_3076.jpg in 7.75 seconds.
Processed face_img_4621.jpg in 0.12 seconds.
Processed face_img_2078.jpg in 3.19 seconds.
Processed flip_face_img_1298.jpg in 4.01 seconds.
Processed face_img_247.jpg in 4.16 seconds.
Processed flip_face_img_3317.jpg in 3.38 seconds.
Processed face_img_656.jpg in 2.93 seconds.
Processed flip_face_img_2627.jpg in 5.42 seconds.
Processed face_img_2733.jpg in 3.41 seconds.
Processed face_img_7313.jpg in 3.44 seconds.
Processed flip_face_bcn_434.jpg in 5.43 seconds.
Processed flip_face_img_1262.jpg in 3.44 seconds.
Process

KeyboardInterrupt: 